# Push to GitHub and Pull Experiments

This tutorial demonstrates how to push experiments managed by MINTO to GitHub Packages using OMMX Artifact functionality, and how other users can pull these experiments. 

OMMX Artifact allows you to push experimental results to other container registries as well. By pushing to GitHub Packages, you can manage both your code and experimental results on GitHub, making it easier to share with your team.

First, let's conduct numerical experiments for mathematical optimization. Here, we will perform the same numerical experiments as in ["Quick Start"](/minto/quickstart).
For details, please refer to the original tutorial.

In [1]:
import minto
import ommx_pyscipopt_adapter as scip_ad
from ommx.dataset import miplib2017

In [2]:
instance_name = "reblock115"
instance = miplib2017(instance_name)

timelimit_list = [0.1, 0.5, 1, 2]

experiment = minto.Experiment("scip_reblock115", auto_saving=False)

experiment.log_instance(instance_name, instance)
adapter = scip_ad.OMMXPySCIPOptAdapter(instance)
scip_model = adapter.solver_input

for timelimit in timelimit_list:
    with experiment.run():
        experiment.log_parameter("timelimit", timelimit)

        # Solve by SCIP
        scip_model.setParam("limits/time", timelimit)
        scip_model.optimize()
        solution = adapter.decode(scip_model)

        experiment.log_solution("scip", solution)

In [3]:
experiment.get_run_table()

solution_scip                                            parameter  \
           objective feasible optimality relaxation start  name timelimit   
run_id                                                                      
0       0.000000e+00     True          0          0  None  scip       0.1   
1       0.000000e+00     True          0          0  None  scip       0.5   
2       0.000000e+00     True          0          0  None  scip       1.0   
3      -2.591821e+07     True          0          0  None  scip       2.0   

       metadata               
         run_id elapsed_time  
run_id                        
0             0     0.106474  
1             1     0.406979  
2             2     0.506567  
3             3     1.007806

## Pushing Results to GitHub
Let's learn how to push your experimental results to GitHub Packages.

### Prerequisites

1. You'll need a GitHub account and repository.
2. If you haven't created one yet, please refer to the GitHub documentation on creating a repository. You'll need to configure OMMX Artifact to access GitHub Packages. While detailed instructions can be found in the OMMX Artifact Notebook, we'll cover the essential setup steps for pushing below.

Pushing to GitHub Packages requires authentication using either a Personal Access Token (PAT) or GITHUB_TOKEN on GitHub Actions. You have two options for authentication:

### Authentication Setup

Pushing to GitHub Packages requires authentication using either a Personal Access Token (PAT) or GITHUB_TOKEN on GitHub Actions. You have two options for authentication:

1. Using the CLI:
    Run ommx login to store authentication information locally on your machine

2. Using Environment Variables:

    - Set the following environment variables:
    ```
    env:
        OMMX_BASIC_AUTH_DOMAIN: ghcr.io
        OMMX_BASIC_AUTH_USERNAME: ${{ github.actor }}
        OMMX_BASIC_AUTH_PASSWORD: ${{ github.token }}
    ```

    - For the `OMMX_BASIC_AUTH_PASSWORD`, you'll need to create a GitHub Personal Access Token. Follow the GitHub guide on creating a Personal Access Token, and make sure to enable the `write:packages` permission.

Note: We use OMMX Artifact functionality for pushing and pulling to/from GitHub Packages. Therefore, GitHub authentication and other settings follow the same configuration as OMMX Artifact. For more details, please refer to [this notebook](https://github.com/Jij-Inc/ommx/blob/main/notebooks/artifact.ipynb).

### Pushing to GitHub Packages
Once your authentication is set up, you can use the Experiment.push_github method to push to GitHub Packages. The method accepts the following parameters:

- org: Your GitHub Organization name
- repo: Your GitHub repository name
- name: The Artifact name (defaults to `Experiment.name`)
- tag: Tag name (defaults to `Experiment.timestamp.strftime('%Y%m%d%H%M%S')`)

In [4]:
# import os

# If environment variables are not set, please use the following to set them.
# Environment variables
# os.environ["OMMX_BASIC_AUTH_DOMAIN"] = 'ghcr.io'
# os.environ["OMMX_BASIC_AUTH_USERNAME"] = 'xxx'
# os.environ["OMMX_BASIC_AUTH_PASSWORD"] = "xxx"

Let's try pushing to GitHub Packages using the `Experiment.push_github` method. As mentioned above, you don't need to specify the name and tag parameters.
The method returns an `ommx.Artifact` object, and you can check the pushed image name using `ommx.Artifact.image_name`.

In [5]:
artifact = experiment.push_github(org="Jij-Inc", repo="MINTO-Public")
artifact.image_name

'ghcr.io/jij-inc/minto-public/scip_reblock115:20241223101904'

## Pull from GitHub

Next, let's try pulling the image we pushed to GitHub Packages. For this purpose, we use the `minto.Experiment.load_from_registry` method. This method supports pulling from other Container Registries as well as GitHub Packages. Therefore, you need to specify the image name directly.

In [8]:
exp2 = minto.Experiment.load_from_registry(
    "ghcr.io/jij-inc/minto-public/scip_reblock115:20241223101904"
)

In [9]:
exp2.get_run_table()

solution_scip                                            parameter  \
           objective feasible optimality relaxation start  name timelimit   
run_id                                                                      
0       0.000000e+00     True          0          0  None  scip       0.1   
1       0.000000e+00     True          0          0  None  scip       0.5   
2       0.000000e+00     True          0          0  None  scip       1.0   
3      -2.591821e+07     True          0          0  None  scip       2.0   

       metadata               
         run_id elapsed_time  
run_id                        
0             0     0.106474  
1             1     0.406979  
2             2     0.506567  
3             3     1.007806

We have introduced how to push and pull to/from GitHub Packages. This method makes it easier to manage experimental results with MINTO on GitHub and share them with your team.